Proyecto Clasificacion Binaria - Carlos Garcia 21000475

### Librerias

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn import svm, tree

In [3]:
from auxiliaryFunctions import getMetrics, writeModelResults, readModels, getModelPredictions, get_topModels, ensemblePredictions, ensembleMetrics

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Enabled compatitility to tf1.x


### Datos

In [5]:
data = pd.read_csv('./input/data_titanic_proyecto.csv')
data.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N


In [6]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data[['passenger_survived']], test_size = 0.2, random_state = 123)

In [7]:
print('Original data: ', data.shape)
print('Xtrain: ', X_train.shape)
print('Xtest: ', X_test.shape)
print('Ytrain: ', y_train.shape)
print('Ytest: ', y_test.shape)

Original data:  (891, 12)
Xtrain:  (712, 11)
Xtest:  (179, 11)
Ytrain:  (712, 1)
Ytest:  (179, 1)


### Preparación de los datos

In [8]:
X_train.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex
329,330,"Hippach, Miss. Jean Gertrude",16.0,0,1,111361,57.9792,B18,C,Upper,F
749,750,"Connaghton, Mr. Michael",31.0,0,0,335097,7.7500,NaN,Q,Lower,M
203,204,"Youseff, Mr. Gerious",45.5,0,0,2628,7.2250,NaN,C,Lower,M
421,422,"Charters, Mr. David",21.0,0,0,A/5. 13032,7.7333,NaN,Q,Lower,M
97,98,"Greenfield, Mr. William Bertram",23.0,0,1,PC 17759,63.3583,D10 D12,C,Upper,M


De todas las columnas disponibles Age, Cabin y Embarked presentan valores nulos. Dado que en las primeras dos columnas mencionadas el porcentaje es mayor a un 5% no es recomendable realizar imputación de los datos, por lo cual serán removidas del dataset. 

In [9]:
X_train.merge(y_train, left_index = True, right_index = True).isna().sum() / len(X_train)

PassengerId           0.000000
Name                  0.000000
Age                   0.207865
SibSp                 0.000000
Parch                 0.000000
Ticket                0.000000
Fare                  0.000000
Cabin                 0.780899
Embarked              0.002809
passenger_class       0.000000
passenger_sex         0.000000
passenger_survived    0.000000
dtype: float64

In [10]:
X_train.dropna(axis = 1, how = 'any', inplace = True)

La codificación de variables categóricas determina las pocas clases presentes tanto para passenger_sex por lo que se usara OHE k-1, en el caso de passenger_class al ser una variable ordinal se aplicara ordinal encoding, lo cual evita la expansión del número de features y toma ventaja del orden mostrado en la variable. 

In [11]:
enc = OrdinalEncoder()
pclass = np.sort(X_train.passenger_class.unique())
pclass = np.expand_dims(pclass, axis = 1)
enc.fit(pclass)

OrdinalEncoder()

In [12]:
X_train['passenger_class'] = enc.transform(X_train[['passenger_class']])
X_train.head()

,PassengerId,Name,SibSp,Parch,Ticket,Fare,passenger_class,passenger_sex
329,330,"Hippach, Miss. Jean Gertrude",0,1,111361,57.9792,2.0,F
749,750,"Connaghton, Mr. Michael",0,0,335097,7.7500,0.0,M
203,204,"Youseff, Mr. Gerious",0,0,2628,7.2250,0.0,M
421,422,"Charters, Mr. David",0,0,A/5. 13032,7.7333,0.0,M
97,98,"Greenfield, Mr. William Bertram",0,1,PC 17759,63.3583,2.0,M


In [13]:
#X_train = pd.get_dummies(X_train, columns = ['passenger_class'])
#X_train.drop(['passenger_class_Upper'], axis = 1, inplace = True)
#X_train.head()

In [14]:
X_train['passenger_sex'].unique()

array(['F', 'M'], dtype=object)

In [15]:
X_train['isFemale'] = np.where(X_train['passenger_sex'] == 'F', 1, 0)

In [16]:
X_train.drop(['PassengerId', 'Name', 'Ticket', 'passenger_sex'], axis = 1, inplace = True)
X_train.head()

,SibSp,Parch,Fare,passenger_class,isFemale
329,0,1,57.9792,2.0,1
749,0,0,7.7500,0.0,0
203,0,0,7.2250,0.0,0
421,0,0,7.7333,0.0,0
97,0,1,63.3583,2.0,0


In [17]:
y_train['passenger_survived'] = np.where(y_train['passenger_survived'] == 'Y', 1, 0)
y_train.head()

,passenger_survived
329,1
749,0
203,0
421,0
97,1


Finalmente se realiza el escalado de las variables para mantenerlas entre 0 y 1, con el fin de facilitar a los algoritmos de GD a converger

In [18]:
scaler = MinMaxScaler()
X_train.iloc[:, :-1] = scaler.fit_transform(X_train.iloc[:, :-1].values)
X_train.head()

,SibSp,Parch,Fare,passenger_class,isFemale
329,0.0,0.2,0.113168,1.0,1
749,0.0,0.0,0.015127,0.0,0
203,0.0,0.0,0.014102,0.0,0
421,0.0,0.0,0.015094,0.0,0
97,0.0,0.2,0.123667,1.0,0


In [19]:
#Train validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 123)

In [20]:
print('Xtrain: ', X_train.shape)
print('Xval: ', X_val.shape)
print('Ytrain: ', y_train.shape)
print('Yval: ', y_val.shape)

Xtrain:  (569, 5)
Xval:  (143, 5)
Ytrain:  (569, 1)
Yval:  (143, 1)


### Preparar datos de prueba

In [21]:
X_test.merge(y_test, left_index = True, right_index = True).isna().sum() / len(X_train)

PassengerId           0.000000
Name                  0.000000
Age                   0.050967
SibSp                 0.000000
Parch                 0.000000
Ticket                0.000000
Fare                  0.000000
Cabin                 0.230228
Embarked              0.000000
passenger_class       0.000000
passenger_sex         0.000000
passenger_survived    0.000000
dtype: float64

In [22]:
X_test.dropna(axis = 1, how = 'any', inplace = True)

In [23]:
X_test['passenger_class'] = enc.transform(X_test[['passenger_class']])
X_test.head()

,PassengerId,Name,SibSp,Parch,Ticket,Fare,Embarked,passenger_class,passenger_sex
172,173,"Johnson, Miss. Eleanor Ileen",1,1,347742,11.1333,S,0.0,F
524,525,"Kassem, Mr. Fared",0,0,2700,7.2292,C,0.0,M
452,453,"Foreman, Mr. Benjamin Laventall",0,0,113051,27.7500,C,2.0,M
170,171,"Van der hoef, Mr. Wyckoff",0,0,111240,33.5000,S,2.0,M
620,621,"Yasbeck, Mr. Antoni",1,0,2659,14.4542,C,0.0,M


In [24]:
#X_test = pd.get_dummies(X_test, columns = ['passenger_class'])
#X_test.drop(['passenger_class_Upper'], axis = 1, inplace = True)
#X_test.head()

In [25]:
X_test['isFemale'] = np.where(X_test['passenger_sex'] == 'F', 1, 0)

In [26]:
X_test.drop(['PassengerId', 'Name', 'Ticket', 'passenger_sex', 'Embarked'], axis = 1, inplace = True)
X_test.head()

,SibSp,Parch,Fare,passenger_class,isFemale
172,1,1,11.1333,0.0,1
524,0,0,7.2292,0.0,0
452,0,0,27.7500,2.0,0
170,0,0,33.5000,2.0,0
620,1,0,14.4542,0.0,0


In [27]:
y_test['passenger_survived'] = np.where(y_test['passenger_survived'] == 'Y', 1, 0)
y_test.head()

,passenger_survived
172,1
524,0
452,0
170,0
620,0


In [28]:
scaler = MinMaxScaler()
X_test.iloc[:, :-1] = scaler.fit_transform(X_test.iloc[:, :-1].values)
X_test.head()

,SibSp,Parch,Fare,passenger_class,isFemale
172,0.2,0.166667,0.044979,0.0,1
524,0.0,0.000000,0.029206,0.0,0
452,0.0,0.000000,0.112112,1.0,0
170,0.0,0.000000,0.135342,1.0,0
620,0.2,0.000000,0.058396,0.0,0


### Definición de Modelos

In [29]:
def train_decisionTree(x, y, criterion = 'entropy', depth = None, min_split = 2):
    
    #Define model training
    clf = tree.DecisionTreeClassifier(criterion = criterion, max_depth = depth, min_samples_split = min_split)
    clf = clf.fit(x.values, y.values)

    #Define configuration string
    features = '_'.join(list(x.columns))
    config_str = 'decisionTree_criterion={}_depth={}_minsplit={}__'.format(criterion, depth, min_split) + features
    
    return clf, config_str
    #tree.plot_tree(clf)

In [30]:
def train_SVM(x, y, c = 1, kernel = 'rbf'):
    
    #Define model training
    clf = svm.SVC(C = c, kernel = kernel)
    clf = clf.fit(x.values, np.squeeze(y.values, axis = 1))

    #Define configuration string
    features = '_'.join(list(x.columns))
    config_str = 'SVM_C={}_kernel={}__'.format(c, kernel) + features
    
    return clf, config_str


In [31]:
def train_NaiveBayes(x, y):
    data = x.merge(y, left_index = True, right_index = True)
    metrics_summary = data.groupby('passenger_survived', as_index = True).agg(['mean', 'std'])
    class_probs = data['passenger_survived'].value_counts(normalize = True).to_frame(name = 'class_prob')
    class_probs.columns = pd.MultiIndex.from_product([class_probs.columns, ['prob']])
    model = metrics_summary.merge(class_probs, left_index = True, right_index = True)

    features = '_'.join(list(x.columns))
    config_str = 'NaiveBayes__' + features

    return model, config_str

In [32]:
#[i for i in range(0, X_train.shape[1])]

In [33]:
def train_LogReg(x, y, lr = 0.1, lambda_reg = 0, regType = 'l2', epochs = 100, batch_size = 1, kprint = 10):
    
    #Get total iterations
    iters = x.shape[0] // batch_size

    #Initialize the graph
    tf.reset_default_graph()

    #Defining tensors and variables
    X = tf.placeholder(dtype = tf.float32, shape = [None, x.shape[1] + 1], name = 'features')
    labels = tf.placeholder(dtype = tf.float32, shape = [None, 1], name = 'class_labels')
    W = tf.get_variable(name = 'weights', shape = [x.shape[1] + 1, 1], dtype = tf.float32, initializer = tf.zeros_initializer())

    #Estimating values
    Z = tf.matmul(X, W, name = 'logit')
    preds = tf.nn.sigmoid(Z, name = 'yhat')

    #Calculating cost function (cross entropy)
    with tf.name_scope('cost_function'):
        if regType == 'l1':
            regularizer = tf.reduce_sum(tf.abs(tf.gather(W, [idx for idx in range(1, x.shape[1] + 1 )])), name = 'regularizer')
        else:
            regularizer = tf.nn.l2_loss(tf.gather(W, [idx for idx in range(1, x.shape[1] + 1 )]))
            
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = labels, logits = Z, name = 'loss'))
        reg_loss = tf.add(loss, lambda_reg * regularizer, name = 'loss_with_reg')

    #Tensorboard scalar summary
    loss_summary = tf.summary.scalar(name = 'CrossEntropy', tensor = reg_loss)

    #Calculate accuracy
    with tf.name_scope('accuracy_definition'):
        correct_prediction = tf.equal(tf.round(preds), labels, name = 'compare_labels')
        accuracy_tensor = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = 'accuracy_metric')

    #Tensorboard scalar accuracy
    accuracy_summary = tf.summary.scalar(name = 'Accuracy', tensor = accuracy_tensor)

    #Gradient and cost error optimization
    with tf.name_scope('optimize_loss'):
        gradients = tf.gradients(reg_loss, [W], name = 'gradients')
        optimizer = tf.assign(W, W - lr * gradients[0], name = 'optimizer')


    with tf.Session() as session:

        #Initialize global vars
        session.run(tf.global_variables_initializer())

        #Reshaping data
        features = '_'.join(list(x.columns))
        ones = np.expand_dims(np.ones_like(x.values[:,0]), axis = 1)
        x = np.hstack((ones, x.values))
        y = y.values

        #Whole batch dictionary
        feed_dict_model = {X:x, labels:y}

        #Define tensorboard writer
        config_str = 'regLog_lr={}_reg={}_lambda={}_mbatch={}_epochs={}__'.format(lr, regType, lambda_reg, batch_size, epochs) + features
        writer = tf.summary.FileWriter('./graphs/' + config_str, session.graph)

        for epoch in range(0, epochs):
            for i in range(0, iters):
                start_sample = i * batch_size
                end_sample = start_sample + batch_size
                x_mb = x[start_sample:end_sample]
                y_mb = y[start_sample:end_sample]

                feed_dict = {X:x_mb, labels:y_mb}
                _, weights = session.run([optimizer, W], feed_dict = feed_dict)

            predictions = session.run([preds], feed_dict = feed_dict_model)
            cross_entropy, lsummary = session.run([reg_loss, loss_summary], feed_dict = feed_dict_model)
            writer.add_summary(lsummary, epoch + 1)

            cpreds, accuracy, asummary = session.run([correct_prediction, accuracy_tensor, accuracy_summary], feed_dict = feed_dict_model)
            writer.add_summary(asummary, epoch + 1)

            if (epoch + 1) % kprint == 0:
                print("Epoch {}: CrossEntropy={} --- Accuracy={}".format(epoch + 1, cross_entropy, accuracy))
            

        #Calculate final model metrics
        _, weights = session.run([optimizer, W], feed_dict = feed_dict_model)
        predictions, cross_entropy = session.run([preds, reg_loss], feed_dict = feed_dict_model)
        cpreds, accuracy = session.run([correct_prediction, accuracy_tensor], feed_dict = feed_dict_model)
        print("Final model: CrossEntropy={} --- Accuracy={}".format(cross_entropy, accuracy))

        #Close tensorboard writer
        writer.close()

        return weights, config_str

### Validación de Modelos y Experimentación

In [34]:
X_train.columns

Index(['SibSp', 'Parch', 'Fare', 'passenger_class', 'isFemale'], dtype='object')

In [ ]:
#X_train = X_train[X_train.columns.difference(['Parch'])]
#X_val = X_val[X_val.columns.difference(['Parch'])]
#X_test = X_test[X_test.columns.difference(['Parch'])]
#X_train.columns

In [36]:
#model, config_str = train_decisionTree(X_train.sample(frac = 0.7, random_state = 1), y_train.sample(frac = 0.7, random_state = 1), depth = 10, criterion = 'entropy')
#model, config_str = train_SVM(X_train.sample(frac = 0.7, random_state = 2), y_train.sample(frac = 0.7, random_state = 2), c = 4, kernel = 'rbf')
#model, config_str = train_NaiveBayes(X_train.sample(frac = 0.7, random_state = 3), y_train.sample(frac = 0.7, random_state = 3),)
model, config_str = train_LogReg(X_train.sample(frac = 0.7, random_state = 4), y_train.sample(frac = 0.7, random_state = 4), lr = 0.01, lambda_reg = 0.0001, regType = 'l1', epochs = 200, batch_size = 10, kprint = 20)

Epoch 20: CrossEntropy=0.6010321378707886 --- Accuracy=0.7587939500808716
Epoch 40: CrossEntropy=0.554241418838501 --- Accuracy=0.7587939500808716
Epoch 60: CrossEntropy=0.5287628173828125 --- Accuracy=0.7613065242767334
Epoch 80: CrossEntropy=0.5139385461807251 --- Accuracy=0.7613065242767334
Epoch 100: CrossEntropy=0.5048167109489441 --- Accuracy=0.7613065242767334
Epoch 120: CrossEntropy=0.49894458055496216 --- Accuracy=0.7613065242767334
Epoch 140: CrossEntropy=0.49502238631248474 --- Accuracy=0.7587939500808716
Epoch 160: CrossEntropy=0.49232274293899536 --- Accuracy=0.7512562870979309
Epoch 180: CrossEntropy=0.4904175400733948 --- Accuracy=0.7487437129020691
Epoch 200: CrossEntropy=0.48904383182525635 --- Accuracy=0.7437185645103455
Final model: CrossEntropy=0.48904213309288025 --- Accuracy=0.7437185645103455


In [ ]:
#tree.plot_tree(model, filled = True)
#plt.show()

In [37]:
#y_pred = getModelPredictions(model, X_val, model_type = 'decisionTree')
#y_pred = getModelPredictions(model, X_val, model_type = 'SVM')
#y_pred = getModelPredictions(model, X_val, model_type = 'NaiveBayes')
y_pred = getModelPredictions(model, X_val, model_type = 'regLog')
y_pred

array([0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0])

In [38]:
model_metrics = getMetrics(config_str, y_val.values, y_pred)
model_metrics

,model,accuracy,error,precision,recall,f1-score
0,regLog_lr=0.01_reg=l1_lambda=0.0001_mbatch=10_...,0.7902,0.2098,0.717,0.717,0.717


In [39]:
model_metrics['model'].values[0]

'regLog_lr=0.01_reg=l1_lambda=0.0001_mbatch=10_epochs=200__SibSp_Parch_Fare_passenger_class_isFemale'

In [40]:
#writeModelResults(model_metrics, model, model_type = 'decisionTree')
#writeModelResults(model_metrics, model, model_type = 'SVM')
#writeModelResults(model_metrics, model, model_type = 'NaiveBayes')
writeModelResults(model_metrics, model, model_type = 'regLog')

Model results saved succesfully


### Selección de modelos

In [41]:
results = pd.read_csv('./results/model_results.csv')
results['model_type'] = results['model'].apply(lambda row: row.split('_', 1)[0])
results.head()

,model,accuracy,error,precision,recall,f1-score,model_type
0,decisionTree_criterion=entropy_depth=None_mins...,0.8042,0.1958,0.7358,0.7358,0.7358,decisionTree
1,decisionTree_criterion=entropy_depth=10_minspl...,0.8112,0.1888,0.7708,0.6981,0.7327,decisionTree
2,decisionTree_criterion=gini_depth=10_minsplit=...,0.7972,0.2028,0.7308,0.7170,0.7238,decisionTree
3,SVM_C=1_kernel=rbf__SibSp_Parch_Fare_passenger...,0.7832,0.2168,0.9231,0.4528,0.6076,SVM
4,SVM_C=2_kernel=rbf__SibSp_Parch_Fare_passenger...,0.8252,0.1748,0.8684,0.6226,0.7253,SVM


In [42]:
top_models = get_topModels(results)
top_models

d:\cgarcia\Galileo\2do_Trimestre\StatisticalLearning_I\project\auxiliaryFunctions.py:149: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  top_models = df.groupby('model_type')['accuracy', 'model'].max()


,accuracy,model
model_type,,
NaiveBayes,0.8112,NaiveBayes__SibSp_Parch_Fare_passenger_class_i...
SVM,0.8322,SVM_C=4_kernel=rbf__SibSp_Parch_Fare_passenger...
decisionTree,0.8112,decisionTree_criterion=gini_depth=10_minsplit=...
regLog,0.8112,regLog_lr=0.1_reg=l2_lambda=0.001_mbatch=20_ep...


### Predicciones

In [43]:
predictions_df = ensemblePredictions(top_models, X_test)
predictions_df.head()

,NaiveBayes,SVM,decisionTree,regLog,mode
0,1,0,1,1,1
1,0,0,0,0,0
2,0,0,0,1,0
3,1,0,1,1,1
4,0,0,0,0,0


In [44]:
model_metrics = ensembleMetrics(predictions_df, y_test)
model_metrics

,model,accuracy,error,precision,recall,f1-score
0,NaiveBayes,0.7877,0.2123,0.6753,0.8000,0.7324
1,SVM,0.8436,0.1564,0.8776,0.6615,0.7544
2,decisionTree,0.8212,0.1788,0.7797,0.7077,0.7419
3,regLog,0.7598,0.2402,0.6375,0.7846,0.7034
4,mode,0.8492,0.1508,0.8393,0.7231,0.7769


### Tensorboard

#### Grafos

<img src='./imgs/train_graph.png'>
<img src='./imgs/predictions_graph.png'>

#### CrossEntropy

<img src='./imgs/cross_entropy.png'>

### Conclusiones

- Agregar sección de conclusiones y recomendaciones (incluyendo opiniones,experiencias ,dificultades y lecciones aprendidas).